In [67]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans

## Reading the tracks and tracks meta as a one dataframe for each.


In [68]:
tracksMetaPath = "00_tracksMeta.csv"    
tracksMeta = pd.read_csv(tracksMetaPath)
counter = tracksMeta["trackId"].max() + 1
for i in range(24):
    if i == 0:
        continue
    if i < 10:
        tracksMetaPath = tracksMetaPath.replace('0'+str(i-1), '0'+str(i))
    if i >= 10: 
        tracksMetaPath = tracksMetaPath.replace(str(i-1), str(i))
    if i == 10: 
        tracksMetaPath = tracksMetaPath.replace('0'+str(i), str(i))
    toJoinFile = pd.read_csv(tracksMetaPath)
    #Give a unique ID for each track who has a previously used  one                    
    for j in range(toJoinFile["trackId"].max() + 1):
        toJoinFile["trackId"].replace({j : counter}, inplace = True)        
        counter = counter + 1
    tracksMeta = pd.concat([tracksMeta, toJoinFile], axis = 0) 

In [69]:
tracksPath = "00_tracks.csv"    
tracks = pd.read_csv(tracksPath)
counter = tracks["trackId"].max() + 1
for i in range(24):
    if i == 0:
        continue
    if i < 10:
        tracksPath = tracksPath.replace('0'+str(i-1), '0'+str(i))
    if i >= 10: 
        tracksPath = tracksPath.replace(str(i-1), str(i))
    if i == 10: 
        tracksPath = tracksPath.replace('0'+str(i), str(i))
    toJoinFile = pd.read_csv(tracksPath)
    #Give a unique ID for each track who has a previously used  one                    
    for j in range(toJoinFile["trackId"].max() + 1):
        toJoinFile["trackId"].replace({j : counter}, inplace = True)        
        counter = counter + 1
    tracks = pd.concat([tracks, toJoinFile], axis = 0) 

In [70]:
#Get all vehicles id
vehicles = tracksMeta.copy()
vehicles = vehicles[vehicles["class"] != 'pedestrian']
vehicles = vehicles[vehicles["class"] != 'motorcycle']
vehicles = vehicles[vehicles["class"] != 'bicycle']

In [72]:
#Get all pedestrian and bicycle id
pede = tracksMeta[tracksMeta["class"] == 'pedestrian'] 
pede = pd.concat([pede,tracksMeta[tracksMeta["class"] == 'bicycle']], axis = 0)

In [75]:
#Get all records for each object
vehiclesData = pd.merge(vehicles['trackId'], tracks)
pedeData = pd.merge(pede['trackId'], tracks)

In [77]:
#Getting the vehicles records that has the same time(frame and recording id) with any of the pedestrians or bicycles
sameFrame = pd.merge(pedeData[['frame', 'recordingId']], vehiclesData)
sameFrame.drop_duplicates(inplace = True)

In [51]:
#Get the minimum and maximun xCenter values for each vehicle to check if any of the VRUs has the same
minXC = sameFrame.groupby(['trackId', 'recordingId'])['xCenter'].min()
maxXC = sameFrame.groupby(['trackId', 'recordingId'])['xCenter'].max() 
#Get tracksID and their number in each record  
recordingSize = sameFrame.groupby(['recordingId'])['trackId'].nunique()
recordingWithTrackId = sameFrame.groupby(['recordingId'])['trackId'].unique()
counter = 0
#vehicleId a list that will contain thee vehicles that has the same xCenter with a pedestrian or a bicycle
vehiclesId = []  
for j in range(23): 
    for k in range(recordingSize.iloc[j]):
        if(pedeData[pedeData['recordingId'] == j ]['xCenter'].between(minXC.iloc[counter], maxXC.iloc[counter]).any()):
            vehiclesId.append(recordingWithTrackId[j][k]) 
        counter = counter + 1 
  

38
2111


In [53]:
#Same process as the above cell
minYC = sameFrame.groupby(['trackId', 'recordingId'])['yCenter'].min()
maxYC = sameFrame.groupby(['trackId', 'recordingId'])['yCenter'].max()  
recordingSize = sameFrame.groupby(['recordingId'])['trackId'].nunique()
recordingWithTrackId = sameFrame.groupby(['recordingId'])['trackId'].unique()
counter = 0
vehiclesIdY = [] 
for j in range(23): 
    for k in range(recordingSize.iloc[j]): 
        if(Pede[Pede['recordingId'] == j ]['yCenter'].between(minYC.iloc[counter], maxYC.iloc[counter]).any()):
            vehiclesIdY.append(recordingWithTrackId[j][k])
        counter = counter + 1 

In [79]:
#getting the intersection between the result from  trackIds that 
vehiclesIds = np.intersect1d(np.array(vehiclesId), np.array(vehiclesIdY) )

In [81]:
#Open the previosly calculated volatility measures for drivers
volatilityMeasures = pd.read_csv("volatilityMeasures.csv")

In [94]:
#Get the volatility measure for each interacted driver 
vehiclesIds = pd.DataFrame({'trackId':vehiclesIds})   
filteredVehicle = pd.merge(volatilityMeasures, vehiclesIds['trackId'])

In [101]:
#Removing an outlier 
filteredVehicle.drop(index = 51, inplace = True)

In [96]:
#drop the trackId column to use the data as an input to the KMeans model 
filteredVehicle.drop(columns = 'trackId', inplace = True)

In [102]:
model = KMeans(n_clusters = 3)
model.fit(filteredVehicle)          

KMeans(n_clusters=3)

In [103]:
secModel = KMeans(n_clusters = 2)
secModel.fit(filteredVehicle) 

KMeans(n_clusters=2)

In [104]:
model.cluster_centers_ 

array([[ 2.37539469e+00,  1.05633847e+00,  8.70753988e+01,
         5.28385647e+01, -6.28239169e+01,  2.03310556e+00,
         3.94207254e-01,  7.11136005e+01,  4.13809056e+01,
        -5.16810035e+01,  2.05598486e+01,  2.52623284e-01,
         0.00000000e+00],
       [ 6.94966278e+00,  8.45703880e-01,  3.22586087e+03,
         6.69808078e+01, -6.24103262e+01,  6.52058952e+00,
         4.39761564e-01,  1.20937194e+03,  5.21845079e+01,
        -3.01139189e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 3.20988417e+00,  1.08173005e+00, -1.58857327e+02,
         6.38684166e+01, -7.31102676e+01,  2.80419350e+00,
         4.16434837e-01, -1.53161422e+02,  4.84157698e+01,
        -5.77138657e+01,  5.93932489e-01,  9.25830455e-01,
         0.00000000e+00]])

In [107]:
secModel.cluster_centers_ 

array([[ 2.70016897e+00,  1.06622060e+00, -8.63895940e+00,
         5.71312638e+01, -6.68272534e+01,  2.33320465e+00,
         4.02857989e-01, -1.61718137e+01,  4.41187987e+01,
        -5.40289283e+01,  1.27893299e+01,  5.14628237e-01,
         0.00000000e+00],
       [ 6.94966278e+00,  8.45703880e-01,  3.22586087e+03,
         6.69808078e+01, -6.24103262e+01,  6.52058952e+00,
         4.39761564e-01,  1.20937194e+03,  5.21845079e+01,
        -3.01139189e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])